In [1]:
from mymodule import MLs
from mymodule import myfunc as mf
import numpy as np
import pandas as pd
import glob
import os
from dotenv import load_dotenv
load_dotenv()
# root_dir = os.environ["root_dir"]
root_dir = "../"

In [2]:
TARGET = "pressure" #ALTIMAGES等のディレクトリにくっつけます
#それぞれ1番最後の'/'は忘れずつけること
ALTIMAGES0 = root_dir + "imgout/0106ML0/"#リコネクションなし分割済データフォルダ.makesepnpy.ipynbであらかじめ分割データを入れておく
ALTIMAGES1 = root_dir + "imgout/0106ML1X/"#リコネクションあり分割済データフォルダ.makesepnpy.ipynbであらかじめ分割データを入れておく
LABEL_SOURCE0 = root_dir + "txt/labels0/"#リコネクションありのラベリングcsvのあるフォルダ
LABEL_SOURCE1 = root_dir + "txt/labels1X/"#リコネクションなしのラベリングcsvのあるフォルダ
IMGSHAPE = [10,100]#出来れば画像サイズはすべて同じで合ってほしい。違うサイズが混じる場合は最も多いサイズを指定すること
DO_PCA = False #データ圧縮。IMGSHAPEと、データサイズが途方もなく多ければTrueにする。
randomstate = None #学習時のパラメータ。初期値はNone。結果を固定したければ1とかに設定する。

ML = MLs.ML(TARGET, ALTIMAGES0, ALTIMAGES1, LABEL_SOURCE0, LABEL_SOURCE1, IMGSHAPE, DO_PCA,randomstate=randomstate)

FileNotFoundError: [Errno 2] No such file or directory: '../imgout/0106ML0/pressure/pressure_77.10.7_40.610.npy'

In [5]:
#学習
#モジュールに書いてるコードで実行
# model = ML.linearSVC()   
# ML.kneighbors()
# ML.rbfSVC()
# ML.XGBoost()

#or
#自由に記述し、MLクラスを利用して実行
from sklearn.svm import LinearSVC
def mylinearSVC(self):    
        model = LinearSVC(C=0.3, random_state=self.RANDOMSTATE) # インスタンスを生成
        model.fit(self.X_train, self.y_train) # モデルの学習
        # 学習データに対する精度
        print("Train :", model.score(self.X_train, self.y_train)) 
        # テストデータに対する精度
        print("Test :", model.score(self.X_test, self.y_test)) 
        print(model.predict(self.X_test))
        pred = model.predict(self.X_test)
        svmres = pd.DataFrame(np.array([self.path_test, self.y_test, pred]).T, columns=["path", "y", "predict"])
        # print(classification_report(self.y_test, pred))
        return model
svm = mylinearSVC(ML)

Train : 0.5757575757575758
Test : 0.42857142857142855
[0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0.]


c:\Users\lkxl1\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [7]:
#参照可能なもの
ML.PATH1
ML.PATH0
ML.ALLTARINDATA0
ML.ALLTARINDATA1
ML.PATH0TEST
ML.PATH1TEST
ML.PATH0TRAIN
ML.PATH1TRAIN
ML.X_test
ML.y_test
ML.path_test
ML.X_train
ML.y_train
ML.path_train
#後から変更可能なもの
ML.randomstate = 1

['../imgout/test1227/magfieldx/magfieldx_49.10.3_47.773.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.10.6_47.773.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.12.13_147.452.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.13.13_42.607.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.7.13_127.625.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.10.4_47.773.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.13.13_102.420.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.10.3_82.620.npy']

In [7]:
#学習モデルをファイルに保存、読み込み例
import pickle
path = "pickletest"

file = open(path, "wb") 
pickle.dump(svm, file)
file.close

with open(path, 'rb') as f:
    dict = pickle.load(f)
print(svm)

LinearSVC(C=0.3)
